### Regression analysis and predictive modeling

This file performs the explanatory regression analysis and the predictive modeling.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.utils import class_weight
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import pickle
from age_gender_distribution_distances import *
sns.set_theme(style="whitegrid", font_scale=3)

In [ ]:
## Define some basic variables
path_figures = "../../Doc/WWW24/Figures/"

# Define party order
party_order = ["linke", "gruene", "spd", "fdp", "union", "afd"]
# Define party labels
party_labels = {"linke": "Linke", "gruene": "Grüne", "spd": "SPD", "fdp": "FDP", "union": "Union", "afd": "AfD"}
# Define color palette
party_pal = {"spd": "#E3000F", "union": "#000000", "gruene": "#46962B", "fdp": "#FFED00", "afd": "#009EE0", "linke": "#990099"}

# Party mapping
party_mapping = {
    "The Left (Germany)": "linke",
    "Green party": "gruene",
    "Alliance '90/The Greens": "gruene",
    "Social Democratic Party of Germany": "spd",
    "Young Socialists in the SPD": "spd",
    "FDP.The Liberals": "fdp",
    "Free Democratic Party (Germany)": "fdp",
    "CDU/CSU": "union",
    "Christian Democratic Union (Germany)": "union",
    "Christian Social Union in Bavaria": "union",
    "Alternative fÃ¼r Deutschland AfD": "afd"
}

In [ ]:
# Importing the dataset
df = pd.read_pickle("../../Data/DE_merged_data.pkl")
df["ad_duration"] = df['ad_duration'].dt.days.astype('int16')

# Filter relevant data
df = df.loc[df["party"] != "others"]

# Replace False and True with NaN and "True"
df = df.replace(False, np.nan)
df = df.replace(True, "True")

# Drop columns with only missing values
df = df.dropna(axis=1, how="all")

In [ ]:
mean_efficiency_party = df.groupby("party").mean("impressions_per_spending")["impressions_per_spending"].reset_index()
mean_efficiency = mean_efficiency_party["impressions_per_spending"].mean()

In [ ]:
# Create age and gender targeting variables
#df['targeting_gender_distribution'] = [get_targeting_gender_distribution(row) for _, row in df.iterrows()]
df['targeting_age_distribution'] = [get_targeting_age_distribution(row) for _, row in df.iterrows()]

# Expand targeting variables to columns
#df_gender = df["targeting_gender_distribution"].apply(pd.Series)
#df_gender.columns = ["targeting_" + c for c in df_gender.columns]

df_age = df["targeting_age_distribution"].apply(pd.Series)
df_age.columns = ["targeting_" + c.replace("-", "_").replace("+", "") for c in df_age.columns]

# Remove original targeting variables
df = df.drop(columns=["targeting_age_distribution"])

# Merge expanded targeting variables
df = pd.concat([df, df_age], axis=1)

#### Variable selection for models

In [ ]:
# Select variable names for regression/prediction model

# Top 10 targeting categories
top10_targeting = ["interests_exclude_use", "interests_include_use", "employers_exclude_use", "exclude_location", "behaviors_exclude_use", "include_lookalike", "exclude_custom_audience", "include_custom_audience", "behaviors_include_use","exclude_lookalike"]

# Include categories
include_targeting_use = [c for c in df.columns if c.endswith("include_use")]  # used "include" category [y/n]
include_targeting_count = [c for c in df.columns if c.endswith("include_count")]  # used n criteria for "include" category
include_other = [c for c in df.columns if c.startswith("include") and "raw" not in c and "location" not in c]  # lookalike, friend connection, audience data missing, custom

# Exclude categories
exclude_targeting_use = [c for c in df.columns if c.endswith("exclude_use")]  # used "exclude" category [y/n]
exclude_targeting_count = [c for c in df.columns if c.endswith("exclude_count")]  # used n criteria for "exclude" category
exclude_other = [c for c in df.columns if c.startswith("exclude") and "raw" not in c]  # lookalike, friend connection, audience data missing, custom

# Demographic targeting
age_targeting_vars = [c for c in df.columns if c.startswith("targeting_") and not "17" in c and "gender" not in c and "age" not in c]
gender_targeting = ["targeting_gender"]

# Political variables
politic_vars = ["party", "candidate_page"]

# Ad variables
ad_vars = ["platform", "ad_duration", "sentiment_class", "weekday_start", "weekday_end"]

In [ ]:
# Create new variable summing up targeting criteria
df["include_count"] = df[include_targeting_count].sum(axis=1)
df["exclude_count"] = df[exclude_targeting_count].sum(axis=1)

#### Explanatory regression analysis

In [ ]:
# Define variable groups

# Define dependent variable
DV = "impressions_per_spending"

# Categorical variables
cat_targeting = include_targeting_use + exclude_targeting_use + include_other + exclude_other # Categorical targeting variables
cat_others = politic_vars + ["platform", "weekday_start", "weekday_end", "sentiment_class", "targeting_gender"] # Other categorical variables

# Continuous variables
#cont_targeting = include_targeting_count + exclude_targeting_count # Continuous targeting variables
cont_others = age_targeting_vars + ["ad_duration", "include_count", "exclude_count"] # Other continuous variables

In [ ]:
# Reformat exclude_other variables
df[exclude_other] = df[exclude_other].replace("True", 1)
df[exclude_other] = df[exclude_other].replace(np.nan, 0)
df[exclude_other] = df[exclude_other].astype(int)

In [ ]:

# Check for variables with no variation and missing values
# Define variables to check
check_vars = cat_targeting + cat_others + cont_others + [DV]

# Check for missing values in variables
print(df[check_vars].isna().sum())
# => 1906 missing values for sentiment variables

# Drop observations with missing values
df = df[check_vars].dropna()

# Check for variables without variation
unique_counts = df[check_vars].nunique()
columns_with_one_unique_value = unique_counts[unique_counts == 1].index.tolist()

print("Columns with only one unique value:")
print(columns_with_one_unique_value)

# Exclude variables without variation
df = df.drop(columns_with_one_unique_value, axis=1)

# Remove variables without variation from variable lists
cat_targeting = [c for c in cat_targeting if c not in columns_with_one_unique_value]
#cont_targeting = [c for c in cont_targeting if c not in columns_with_one_unique_value]

In [ ]:
# Create dummies for categorical variables

# List of categorical variables
cat_vars = cat_targeting + cat_others

# Encode dummy variables for categorical variables
model_df = pd.get_dummies(df, columns=cat_vars, drop_first=False)

##### Ordinary Least Squares (OLS) regression

In [ ]:
# List of dummy for targeting variables
dummy_targeting = [c + str("_1") for c in cat_targeting]
# List of dummies for Top 10 targeting variables
dummy_targeting_top10 = [c + str("_1") for c in top10_targeting]
# List of dummies for other categorical targeting variables
dummy_targeting_others = [c for c in model_df.columns if c not in dummy_targeting and c.startswith("include_") and not c.endswith("raw") and not c.endswith("location") and not c.endswith("_0")]
# List of dummies for weekday variables
dummy_weekday = [c for c in model_df.columns if c.startswith("weekday_") and not c.endswith("_0") and "end" not in c] # Monday is reference category
# List of gender dummies
dummy_gender = ["targeting_gender_Men", "targeting_gender_Women"] # Targeting all genders is reference category
# List of party dummies
dummy_party = [c for c in model_df.columns if c.startswith("party_") and not c.endswith("afd")] # AfD is reference category
# List of sentiment dummies
dummy_sentiment = ["sentiment_class_positive", "sentiment_class_negative"] # Neutral sentiment is reference category
# List of platform dummies
dummy_platform = ["platform_1", "platform_2"] # 1 = Facebook, 2 = Instagram, 3 = Both platforms (Reference category)

In [ ]:
# Standardize continuous variables
cont_std = (model_df[cont_others] - model_df[cont_others].mean()) / model_df[cont_others].std()
cont_std.columns = [c + "_std" for c in cont_others]
model_df = pd.concat([model_df, cont_std], axis=1)
cont_others_std = [c + "_std" for c in cont_others]

In [ ]:
# Define variables for OLS models

# Model 1: Targeting variables
vars_model_targeting = dummy_targeting_top10 + ["include_count_std", "exclude_count_std"]

# Model 2: Demographic targeting variables
vars_model_demographics = dummy_gender + [c + "_std" for c in age_targeting_vars]

# Model 3: Ad characteristics
vars_model_ads = dummy_weekday + dummy_party + dummy_sentiment + dummy_platform + ["ad_duration_std", "candidate_page_1"]

In [ ]:
# Function to perform OLS regression
def regression_analysis(data, variables, filename=None):
    # Create formula for OLS model
    variables_str = " + ".join(variables)
    y, X = dmatrices(DV + ' ~ ' + variables_str, data=data, return_type='dataframe')
    
    # Fit model
    mod = sm.OLS(y, X)    # Describe model
    res = mod.fit(cov_type='HC3')       # Fit model
    
    # Save results
    res.save("../../Data/Models/OLS/" + "OLS_" + filename + ".pkl")
    
    # Print summary
    print(res.summary())
    
    # Print goodness of fit
    print("Goodness of fit")
    print("N = "+str(res.nobs))
    print("R2 = "+str(res.rsquared))
    print("R2 adj = "+str(res.rsquared_adj))
    print("MSE = "+str(res.mse_model))

In [ ]:
# OLS regression for targeting variables
models = [vars_model_targeting, vars_model_demographics, vars_model_ads]
names = ["targeting", "demographic", "ads"]

for model, file in zip(models, names):
    regression_analysis(model_df, model, filename=file)

### Predictive modeling

#### Define model variables

In [ ]:
# Define independent variables (We have to re-do this because we dropped a reference category for the OLS model)
# List of dummy for targeting variables
dummy_targeting = [c + str("_0") for c in cat_targeting] + [c + str("_1") for c in cat_targeting]
# List of dummies for weekday variables
dummy_weekday = [c for c in model_df.columns if c.startswith("weekday_start")] # 0 = Monday, 6 = Sunday
# List of gender dummies
dummy_gender = ["targeting_gender_Men", "targeting_gender_Women", "targeting_gender_All"]
# List of party dummies
dummy_party = [c for c in model_df.columns if c.startswith("party_")]
# List of sentiment dummies
dummy_sentiment = ["sentiment_class_positive", "sentiment_class_neutral", "sentiment_class_negative"]
# List of candidate page dummies
dummy_candidate_page = ["candidate_page_0", "candidate_page_1"]
# List of platform dummies
dummy_platform = ["platform_1", "platform_2", "platform_3"] # 1 = Facebook, 2 = Instagram, 3 = Both platforms

In [ ]:
# Define independent variables for different models
covariates = dummy_weekday + dummy_gender + dummy_party + dummy_sentiment + dummy_candidate_page + dummy_platform + cont_others

# Model 1: Categorical targeting variables
vars_model_cat = dummy_targeting + covariates

In [ ]:
# Convert party dummies to single party variable
model_df["party"] = model_df[dummy_party].idxmax(axis=1).str.replace("party_", "")
# Count number of observations
model_df["party"].value_counts()

#### Set up preprocessing pipeline and models

In [ ]:
# Set up preprocessing pipeline

# Scale numerical variables
# Create a ColumnTransformer to specify how to preprocess each column
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), cont_others)  # Scale numerical columns
    ],
    remainder='passthrough',  # Keep non-numerical columns as they are
    verbose_feature_names_out=False)

# Create a pipeline to apply the preprocessor
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
# Set up models
# Initialize the random forest regressor
rf_regressor = RandomForestRegressor()
# Initialize the XGBoost regressor
xgb_regressor = xgb.XGBRegressor()

# Define a grid of hyperparameters to search over
param_grid_xgb = {
    'n_estimators': [100, 200, 300],  # Number of boosting rounds
    'learning_rate': [0.3, 0.4],  # Step size shrinkage to prevent overfitting
    'max_depth': [3, 4, 5],  # Maximum depth of the tree
    'subsample': [0.8, 0.9, 1.0],  # Fraction of samples used for fitting trees
    'colsample_bytree': [0.8, 0.9, 1.0],  # Fraction of features used for fitting trees
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_features': ['sqrt', 'log2'],  # Number of features to consider at each split
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'bootstrap': [True, False],  # Whether to bootstrap samples when building trees
}

#### Set up and split data

In [ ]:
# Set a random seed for reproducibility
np.random.seed(42)

In [ ]:
# Set number of runs
runs = 10

In [ ]:
# Create list of seed values
seeds = np.random.randint(1, 1000, size=runs)

In [ ]:
# Define dependent variable
y = model_df["impressions_per_spending"]
# Define independent variables
X = model_df[vars_model_cat]

In [ ]:
# Create list with 10 different train/test splits
x_train_list = []
x_test_list = []
y_train_list = []
y_test_list = []

for seed in seeds:
    # Create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    
    # Preprocess data
    # Fit the pipeline to the training data
    X_train_scaled = pd.DataFrame(pipeline.fit_transform(X_train))
    # Apply the pipeline to the test data
    X_test_scaled = pd.DataFrame(pipeline.transform(X_test))
    
    # Assign column names
    X_train_scaled.columns = pipeline.get_feature_names_out()
    X_test_scaled.columns = pipeline.get_feature_names_out()
    
    # Append input and results to list
    x_train_list.append(X_train_scaled)
    x_test_list.append(X_test_scaled)
    y_train_list.append(y_train)
    y_test_list.append(y_test)
    

In [ ]:
# Function to train, fit, and evaluate model
def model_training(model, param_grid, x_train, y_train, x_test, y_test):
    
    # Create sample weights to handle class imbalance
    party_train = x_train[dummy_party].idxmax(axis=1).str.replace("party_", "")
    weights_train = class_weight.compute_sample_weight(class_weight="balanced", y=party_train)
    
    # Initialize GridSearchCV for hyperparameter tuning with cross-validation
    grid_search = GridSearchCV(estimator=model, 
                               param_grid=param_grid,
                               scoring='neg_mean_squared_error', 
                               cv=10,
                               n_jobs=30)

    # Fit the grid search to the training data
    grid_search.fit(x_train, y_train, sample_weight=weights_train)

    # Get the best hyperparameters from the search
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)

    # Use the best model from the grid search
    best_model = grid_search.best_estimator_

    # Fit the best model to the training data
    best_model.fit(x_train, y_train, sample_weight=weights_train)

    # Evaluate the model on the test data
    y_pred = best_model.predict(x_test)

    # Compute mean absolute value (MAE)
    mae = np.mean(abs(y_test - y_pred))

    # Compute the MSE
    mse = mean_squared_error(y_test, y_pred)

    # Compute the RMSE
    rmse = np.sqrt(mse)

    # Compute the R^2 score
    r2 = r2_score(y_test, y_pred)
    
    metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}
    
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"Root Mean Squared Error: {rmse}")
    print(f"R-squared: {r2}")
    
    return best_params, best_model, metrics  

#### Train, fit, and evaluate model

In [ ]:
# Set up list of results
# XGBoost
best_params_xgb_list = []
best_model_xgb_list = []
metrics_xgb_list = []

# Random Forest
best_params_rf_list = []
best_model_rf_list = []
metrics_rf_list = []

In [ ]:
# Train, fit, and evaluate model for 10 different seeds
# XGBoost
for run in range(runs):
    print("Run:", run)
    best_params_xgb, best_model_xgb, metrics_xgb = model_training(xgb_regressor, param_grid_xgb, x_train=x_train_list[run], y_train=y_train_list[run], x_test=x_test_list[run], y_test=y_test_list[run])
    
    # Append input and results to list
    best_params_xgb_list.append(best_params_xgb)
    best_model_xgb_list.append(best_model_xgb)
    metrics_xgb_list.append(metrics_xgb)
    
# Save results
pickle.dump(best_params_xgb_list, open("../../Data/Models/ML/xgb_best_params.pkl", 'wb'))
pickle.dump(best_model_xgb_list, open("../../Data/Models/ML/xgb_best_model.pkl", 'wb'))
pickle.dump(metrics_xgb_list, open("../../Data/Models/ML/xgb_metrics.pkl", 'wb'))

In [ ]:
# Random Forest
for run in range(runs):
    print("Run:", run)
    best_params_rf, best_model_rf, metrics_rf = model_training(rf_regressor, param_grid_rf, x_train=x_train_list[run], y_train=y_train_list[run], x_test=x_test_list[run], y_test=y_test_list[run])
    
    # Append input and results to list
    best_params_rf_list.append(best_params_rf)
    best_model_rf_list.append(best_model_rf)
    metrics_rf_list.append(metrics_rf)
    
# Save results
pickle.dump(best_params_rf_list, open("../../Data/Models/ML/rf_best_params.pkl", 'wb'))
pickle.dump(best_model_rf_list, open("../../Data/Models/ML/rf_best_model.pkl", 'wb'))
pickle.dump(metrics_rf_list, open("../../Data/Models/ML/rf_metrics.pkl", 'wb'))

#### Analysis of residuals

In [ ]:
# Compute avg. metrics
metrics_xgb_avg = {"mae": np.mean([m["mae"] for m in metrics_xgb_list]), "mse": np.mean([m["mse"] for m in metrics_xgb_list]), "rmse": np.mean([m["rmse"] for m in metrics_xgb_list]), "r2": np.mean([m["r2"] for m in metrics_xgb_list])}
metrics_rf_avg = {"mae": np.mean([m["mae"] for m in metrics_rf_list]), "mse": np.mean([m["mse"] for m in metrics_rf_list]), "rmse": np.mean([m["rmse"] for m in metrics_rf_list]), "r2": np.mean([m["r2"] for m in metrics_rf_list])}

# Compute std. metrics
metrics_xgb_std = {"mae": np.std([m["mae"] for m in metrics_xgb_list]), "mse": np.std([m["mse"] for m in metrics_xgb_list]), "rmse": np.std([m["rmse"] for m in metrics_xgb_list]), "r2": np.std([m["r2"] for m in metrics_xgb_list])}
metrics_rf_std = {"mae": np.std([m["mae"] for m in metrics_rf_list]), "mse": np.std([m["mse"] for m in metrics_rf_list]), "rmse": np.std([m["rmse"] for m in metrics_rf_list]), "r2": np.std([m["r2"] for m in metrics_rf_list])}

# Select best model based on RMSE
if metrics_rf_avg["rmse"] < metrics_xgb_avg["rmse"]:
    best_model = best_model_rf_list
    print("Best model: Random forest")
else:
    best_model = best_model_xgb_list
    print("Best model: XGBoost")

In [ ]:
# Compute residuals for each run by party
residuals = []
residuals_avg = []

for run in range(runs):
    print("Run:", run)
    # Compute residuals
    residuals_run = pd.Series(y_test_list[run] - best_model[run].predict(x_test_list[run])).reset_index(drop=True)
    # Combine residuals with party
    party = x_test_list[run][dummy_party].idxmax(axis=1).str.replace("party_", "")
    residuals_run = pd.concat([residuals_run, party], axis=1)
    
    # Average residuals per party
    residuals_run_avg = residuals_run.groupby(0).mean().reset_index()
    residuals_run_avg.columns = ["party", "residuals"]
    
    # Append results
    residuals.append(residuals_run)
    residuals_avg.append(residuals_run_avg)

In [ ]:
# Flatten list of residuals_avg to dataframe with indicator for run
residuals = pd.concat(residuals_avg, axis=0).reset_index(drop=True)

# Compute mean residuals per party over all runs
avg_residuals = residuals.groupby("party").mean().reset_index()
# Compute std. residuals per party over all runs
std_residuals = residuals.groupby("party").std().reset_index()
# Combine mean and std. residuals
residuals_plot = pd.merge(avg_residuals, std_residuals, on="party", suffixes=("_avg", "_std"))

In [ ]:
# Compute relative advantage
residuals_plot["residuals_avg"] / mean_efficiency

In [ ]:
residuals_plot.to_pickle("../../Data/Models/ML/residuals_plot.pkl")